In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pkg_resources 
from tdev2.readers.gold_reader import *
from tdev2.readers.disc_reader import *

In [3]:
from tdev2.measures.ned import *
from tdev2.measures.boundary import *
from tdev2.measures.grouping import *
from tdev2.measures.coverage import *
from tdev2.measures.token_type import *

In [4]:
def prf2dict(dct, measurename, obj):
    # save precision-recall-f of tde objects to dictionary
    dct[measurename + '_P'] = obj.precision
    dct[measurename + '_R'] = obj.recall
    dct[measurename + '_F'] = obj.fscore
    
    return dct


def compute_scores(gold, disc, measures=[]):
    scores = dict()
    
    # Launch evaluation of each metric
    if len(measures) == 0 or "boundary" in measures:
        print('Computing Boundary...')
        boundary = Boundary(gold, disc)
        boundary.compute_boundary()
        scores = prf2dict(scores, 'boundary', boundary)
        
    if len(measures) == 0 or "grouping" in measures:
        print('Computing Grouping...')
        grouping = Grouping(disc)
        grouping.compute_grouping()
        scores = prf2dict(scores, 'grouping', grouping)    
        
    if len(measures) == 0 or "token/type" in measures:
        print('Computing Token and Type...')
        token_type = TokenType(gold, disc)
        token_type.compute_token_type()
        scores['token_P'],scores['token_R'],scores['token_F'] = token_type.precision[0], token_type.recall[0], token_type.fscore[0]
        scores['type_P'],scores['type_R'],scores['type_F'] = token_type.precision[1], token_type.recall[1], token_type.fscore[1]        
        
    if len(measures) == 0 or "coverage" in measures:
        print('Computing Coverage...')
        coverage = Coverage(gold, disc)
        coverage.compute_coverage()
        scores['coverage'] = coverage.coverage

        
    if len(measures) == 0 or "coverageNS" in measures:
        print('Computing Coverage No Single...')
        coverage = Coverage_NoSingleton(gold, disc)
        coverage.compute_coverage()
        scores['coverageNS'] = coverage.coverage

        
    if len(measures) == 0 or "ned" in measures:
        print('Computing NED...')
        ned = Ned(disc)
        ned.compute_ned()
        scores['ned'] = ned.ned
        
        
    
    return scores

In [11]:
os.chdir('build/lib/')

In [18]:
os.chdir('../../')

In [5]:
import sys
sys.path.append('/home/korhan/Dropbox/tez_scripts/')
from tdev2.utils import zrexp2tde, sdtw2tde, narrow_gold, read_config

wrd_path = 'tdev2/share/phoenix.wrd'
phn_path = 'tdev2/share/phoenix.phn'
cnf_path = 'config.json'

# read_config(cnf_path)

# exp_path = '/home/korhan/Desktop/zerospeech2017/track2/src/ZRTools/exp/'
exp_path = '/home/korhan/Desktop/tez/sdtw_exps/sdtw_pairs_A_0_341_c3right_PCA_Wh_diag_olapFalse_w8_L2_extend_r0.3_diag_thr0.7_loss_funceuclid_end_cut0/postpairwise_cost0.001_olap0.25/'

disc_clsfile = sdtw2tde(exp_path)

print(disc_clsfile)

/home/korhan/Desktop/tez/sdtw_exps/sdtw_pairs_A_0_341_c3right_PCA_Wh_diag_olapFalse_w8_L2_extend_r0.3_diag_thr0.7_loss_funceuclid_end_cut0/postpairwise_cost0.001_olap0.25/master_graph.class


In [6]:
from tdev2.eval_sign import try_compute_scores

In [10]:
kwargs = {'config_file' : 'config.json',
         'njobs': 1}

In [14]:
try_compute_scores(gold, disc, measures=[], **kwargs)

Computing Boundary...
Computing Grouping...
Number of grouping jobs: 1
Computing Token and Type...
Computing Coverage...
Computing Coverage...
Computing Coverage No Single...
*** Config file read, ovth 5.0 ***
Computing NED...


{'boundary_P': 0.2656,
 'boundary_R': 0.0003,
 'boundary_F': 0.0005,
 'n_clus': 39,
 'n_node': 40,
 'grouping_P': 1.0,
 'grouping_R': 0.0952,
 'grouping_F': 0.1738,
 'token_P': 0.1389,
 'token_R': 0.0001,
 'token_F': 0.0002,
 'type_P': 0.2143,
 'type_R': 0.0024,
 'type_F': 0.0048,
 'coverage': 0.0002,
 'coverageNS': 0.0002,
 'coverageNS_f': 0.0001,
 'ned': 1.0}

In [11]:
# read_config(cnf_path)
gold = Gold(wrd_path=wrd_path, phn_path=phn_path, **kwargs) 
disc = Disc(disc_clsfile, gold) 


*** Config file read ***


In [10]:
disc_clsfile

'/home/korhan/Desktop/tez/sdtw_exps/sdtw_pairs_A_0_341_c3right_PCA_Wh_diag_olapFalse_w8_L2_extend_r0.3_diag_thr0.7_loss_funceuclid_end_cut0/postpairwise_cost0.001_olap0.25/master_graph.class'

In [25]:
scores = compute_scores(gold, disc)
scores['n_clus'] = len(disc.clusters)
scores['n_node'] = len(disc.intervals)
scores

Computing Boundary...
Computing Grouping...
Number of grouping jobs: 1
Computing Token and Type...
Computing Coverage...
Computing Coverage No Single...
Computing NED...


{'boundary_P': 0.5131578947368421,
 'boundary_R': 0.005875086619866831,
 'boundary_F': 0.011594830287040826,
 'grouping_P': 0.5307443365695792,
 'grouping_R': 0.45054945054945056,
 'grouping_F': 0.48687383012976415,
 'token_P': 0.20609579100145137,
 'token_R': 0.0023389501078881917,
 'token_F': 0.004625407166123778,
 'type_P': 0.4444444444444444,
 'type_R': 0.02924451665312754,
 'type_F': 0.054878048780487805,
 'coverage': 0.005818596691386195,
 'coverageNS': 0.006113361801525603,
 'ned': 0.5161290322580645,
 'n_clus': 521,
 'n_node': 689}

### write read config file

In [28]:
conf = {'overlap_th': 5. ,
'excluded_units' : ['SIL','__ON__','__OFF__','__EMOTION__','SPN'] ,
'discoverable_th' : 1}
conf

{'overlap_th': 5.0,
 'excluded_units': ['SIL', '__ON__', '__OFF__', '__EMOTION__', 'SPN'],
 'discoverable_th': 1}

In [29]:
import json
config_pth = 'config.json'

with open(config_pth, 'w') as f:
    json.dump(conf, f)

In [32]:
with open(config_pth, 'r') as f:
    conf = json.load(f)
    
conf['overlap_th']

5.0